In [1]:
import sys

import houndify

In [2]:
import ffmpeg

In [3]:
import pkg_resources

In [4]:
sys.path.insert(1, '../speech-driven-animation/')

In [5]:
import sda

In [6]:
import torch

In [11]:
import os
import base64
import time
import wave


In [13]:
CLIENT_ID = "Rco-d2ARwyAUSvklEH_ePg=="
CLIENT_KEY = "V6TzSgIOkx0CW4AkBSxgLNrcHMEZ26PSB4uLaG-n8t6PsjZSEzBF-aGfXQv-mWXha0_grVB-TO5MGdKhx9XNFA=="
AUDIO_FILE = "whattimeisitindenver.wav"

BUFFER_SIZE = 256

audio = wave.open(AUDIO_FILE)
if audio.getsampwidth() != 2:
  print("%s: wrong sample width (must be 16-bit)" % fname)
  sys.exit()
if audio.getframerate() != 8000 and audio.getframerate() != 16000:
  print("%s: unsupported sampling frequency (must be either 8 or 16 khz)" % fname)
  sys.exit()
if audio.getnchannels() != 1:
  print("%s: must be single channel (mono)" % fname)
  sys.exit()


audio_size = audio.getnframes() * audio.getsampwidth()
audio_duration = audio.getnframes() / audio.getframerate()
chunk_duration = BUFFER_SIZE * audio_duration / audio_size


#
# Simplest HoundListener; just print out what we receive.
# You can use these callbacks to interact with your UI.
#
class MyListener(houndify.HoundListener):
  def onPartialTranscript(self, transcript):
    print("Partial transcript: " + transcript)
  def onFinalResponse(self, response):
    print("Final response: " + str(response))
    
    response_bytes = response["AllResults"][0]["ResponseAudioBytes"]
    os.system("say {}".format(response_text)) 
    print("Response bytes:" + response_bytes)
    print("Final response: " + str(response))

    # convert bytes string to binary
    decoded = base64.decodebytes(response_bytes.encode("ascii"))

    # writes response to wav file
    with open('myfile.wav', mode='wb') as f:
      f.write(decoded)
    
    va = sda.VideoAnimator(gpu=0, model_path="timit")# Instantiate the animator
    vid, aud = va("IMG_9997.bmp", "sabrina_audio.wav")
    va.save_video(vid, aud, "generated_sabrina.mp4")
    
    # open video
      
    
  def onError(self, err):
    print("Error: " + str(err))


client = houndify.StreamingHoundClient(CLIENT_ID, CLIENT_KEY, "test_user", enableVAD=False)
client.setLocation(37.388309, -121.973968)
client.setSampleRate(audio.getframerate())
client.setHoundRequestInfo("ResponseAudioVoice", "Sharon")
client.setHoundRequestInfo("ResponseAudioShortOrLong", "Short")


client.start(MyListener())

while True:
  chunk_start = time.time()

  samples = audio.readframes(BUFFER_SIZE)
  if len(samples) == 0: break
  if client.fill(samples): break

  # Uncomment the line below to simulate real-time request
  # time.sleep(chunk_duration - time.time() + chunk_start) 

result = client.finish() # returns either final response or error



Partial transcript: 
Partial transcript: 
Partial transcript: 
Partial transcript: 
Partial transcript: 
Partial transcript: 
Partial transcript: 
Partial transcript: 
Partial transcript: what time
Partial transcript: what time is
Partial transcript: what time is
Partial transcript: what time is it
Partial transcript: what time is it in
Partial transcript: what time is it in
Partial transcript: what time is it in de
Partial transcript: what time is it in denver
Partial transcript: what time is it in denver
Partial transcript: what time is it in denver
Partial transcript: what time is it in denver
Partial transcript: what time is it in denver
Partial transcript: what time is it in denver
Partial transcript: what time is it in denver
Partial transcript: what time is it in denver
Partial transcript: what time is it in denver
Partial transcript: what time is it in denver
Final response: {'BuildInfo': {'Variant': 'release', 'User': 'knightly', 'BuildNumber': '6107', 'Kind': 'Low Fat', 'SVNR

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/anaconda/envs/py35/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/anaconda/envs/py35/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "/data/home/friend/notebooks/houndify.py", line 362, in _callback
    listener.onFinalResponse(parsedMsg)
  File "<ipython-input-13-f468c31ff162>", line 34, in onFinalResponse
    response_bytes = response["AllResults"][0]["ResponseAudioBytes"]
KeyError: 'ResponseAudioBytes'



In [8]:
CLIENT_ID = "Rco-d2ARwyAUSvklEH_ePg=="
CLIENT_KEY = "V6TzSgIOkx0CW4AkBSxgLNrcHMEZ26PSB4uLaG-n8t6PsjZSEzBF-aGfXQv-mWXha0_grVB-TO5MGdKhx9XNFA=="
BUFFER_SIZE = 512

#
# Simplest HoundListener; just print out what we receive.
# You can use these callbacks to interact with your UI.
#
class MyListener(houndify.HoundListener):
  def onPartialTranscript(self, transcript):
    print("Partial transcript: " + transcript)
  def onFinalResponse(self, response):
    # Playing the converted file 
    response_text = response["AllResults"][0]["SpokenResponse"]
    # get response audio bytes string
    response_bytes = response["AllResults"][0]["ResponseAudioBytes"]
    os.system("say {}".format(response_text)) 
    print("Response bytes:" + response_bytes)
    print("Final response: " + str(response))

    # convert bytes string to binary
    decoded = base64.decodebytes(response_bytes.encode("ascii"))

    # writes response to wav file
    with open('myfile.wav', mode='wb') as f:
      f.write(decoded)
    
#     va = sda.VideoAnimator(gpu=0, model_path="timit")# Instantiate the animator
#     vid, aud = va("IMG_9997.bmp", "sabrina_audio.wav")
#     va.save_video(vid, aud, "generated_sabrina.mp4")
      
  
  def onError(self, err):
    print("Error: " + str(err))


client = houndify.StreamingHoundClient(CLIENT_ID, CLIENT_KEY, "test_user")
client.setLocation(37.388309, -121.973968)
client.setHoundRequestInfo("ResponseAudioVoice", "Sharon")
client.setHoundRequestInfo("ResponseAudioShortOrLong", "Short")

## Uncomment the lines below to see an example of using a custom
## grammar for matching.  Use the file 'turnthelightson.wav' to try it.
# clientMatches = [ {
#   "Expression" : '([1/100 ("can"|"could"|"will"|"would")."you"].[1/10 "please"].("turn"|"switch"|(1/100 "flip"))."on".["the"].("light"|"lights").[1/20 "for"."me"].[1/20 "please"])|([1/100 ("can"|"could"|"will"|"would")."you"].[1/10 "please"].[100 ("turn"|"switch"|(1/100 "flip"))].["the"].("light"|"lights")."on".[1/20 "for"."me"].[1/20 "please"])|((("i".("want"|"like"))|((("i".["would"])|("i\'d")).("like"|"want"))).["the"].("light"|"lights").["turned"|"switched"|("to"."go")|(1/100"flipped")]."on".[1/20"please"])"',
#   "Result" : { "Intent" : "TURN_LIGHT_ON" },
#   "SpokenResponse" : "Ok, I\'m turning the lights on.",
#   "SpokenResponseLong" : "Ok, I\'m turning the lights on.",
#   "WrittenResponse" : "Ok, I\'m turning the lights on.",
#   "WrittenResponseLong" : "Ok, I\'m turning the lights on."
# } ]
# client.setHoundRequestInfo('ClientMatches', clientMatches)

client.start(MyListener())

while True:
  samples = sys.stdin.buffer.read(BUFFER_SIZE)
  if len(samples) == 0: break
  if client.fill(samples): break

client.finish()

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/anaconda/envs/py35/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/anaconda/envs/py35/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "/data/home/friend/notebooks/houndify.py", line 362, in _callback
    listener.onFinalResponse(parsedMsg)
  File "<ipython-input-8-feb52d76c9a9>", line 14, in onFinalResponse
    response_text = response["AllResults"][0]["SpokenResponse"]
KeyError: 'AllResults'



{'AudioLength': 0,
 'BuildInfo': {'BuildNumber': '6107',
  'Date': 'Sun Dec  8 20:17:31 PST 2019',
  'Kind': 'Low Fat',
  'Machine': '7fhpjh2.pnp.melodis.com',
  'SVNBranch': 'dev_multi_machine',
  'SVNRevision': '54997',
  'User': 'knightly',
  'Variant': 'release'},
 'DomainUsage': [],
 'Format': 'SoundHoundVoiceSearchResult',
 'FormatVersion': '1.0',
 'NumToReturn': 0,
 'QueryID': '9a2e5e70-dc69-f817-52d2-d7e5b2e53120',
 'ServerGeneratedId': '9a2e5e70-dc69-f817-52d2-d7e5b2e53120',
 'Status': 'OK'}

/bin/sh: 1: ./sample_stdin.py: not found

Input File     : 'default' (alsa)
Channels       : 2
Sample Rate    : 48000
Precision      : 16-bit
Sample Encoding: 16-bit Signed Integer PCM

In:0.00% 00:00:00.26 [00:00:00.00] Out:8.19k [      |      ]        Clip:0    sox FAIL sox: `-' error writing output file: Broken pipe
In:0.00% 00:00:00.43 [00:00:00.00] Out:16.4k [      |      ]        Clip:0    rec FAIL sox: `-' error writing output file: Broken pipe
Done.


In [18]:
va = sda.VideoAnimator(gpu=0, model_path="timit")# Instantiate the animator

In [22]:
vid, aud = va("IMG_9997.bmp", "sabrina_audio.wav")

/anaconda/envs/py35/lib/python3.5/site-packages/scipy/signal/signaltools.py:2223: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Y[sl] = X[sl]
/anaconda/envs/py35/lib/python3.5/site-packages/scipy/signal/signaltools.py:2225: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Y[sl] = X[sl]
/anaconda/envs/py35/lib/python3.5/site-packages/scipy/signal/signaltools.py:2233: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an a

In [23]:
va.save_video(vid, aud, "generated_sabrina.mp4")

In [ ]:
!pip install conda

In [ ]:
!conda install -c conda-forge ffmpeg-python